In [10]:
import datetime
import struct
import numpy as np

In [83]:
def gen_Zapit_byte_tuple(trial_state_command, arg_keys_dict, arg_values_dict):
    """
    Generate a byte tuple to communicate with Zapit device.

    Parameters:
    trial_state_command (int): Integer value representing the trial state command.
    arg_keys_dict (dict): A dictionary specifying the channels through Zapit is being communicated with.
    arg_values_dict (dict): A dictionary specifying the actual boolean or integer values being communicated to Zapit.

    Returns:
    Tuple: A tuple containing two byte tuples. The first byte tuple contains the state command byte, the argument keys byte,
    the argument values byte, and the condition number byte (if applicable). The second byte tuple contains the integer
    values of the state command, argument keys, argument values, and condition number (if applicable).
    """
    # map arg_keys_dict to ints
    keys_to_int_dict = {"conditionNum_channel": 1, "laser_channel": 2, "hardwareTriggered_channel": 4,
                        "logging_channel": 8, "verbose_channel": 16}

    # map arg_values_dict to ints
    bool_values_to_int_dict = {"conditionNum": 1, "laser_ON": 2, "hardwareTriggered_ON": 4,
                               "logging_ON": 8, "verbose_ON": 16}

    # map True/False boolean values to 1/0
    bool_to_int_dict = {True: 1, False: 0}

    # sum arg_keys ints and convert to byte
    arg_keys_int = 0
    for arg, key in arg_keys_dict.items():
        arg_keys_int += bool_to_int_dict[key] * keys_to_int_dict[arg]
        arg_keys_byte = arg_keys_int.to_bytes(1, 'big')

    # sum arg_value ints and convert to byte
    arg_values_int = 0
    for arg, value in arg_values_dict.items():
        try:
            arg_values_int += bool_to_int_dict[value] * bool_values_to_int_dict[arg]
            arg_values_byte = arg_values_int.to_bytes(1, 'big')
        except:
            pass

    # define trial states where python will query zapit
    trial_state_commands_dict = {"stimConfLoaded": 2, "return_state": 3,
                                 "numCondition": 4, "sendsamples": 1, "stopoptostim": 0}
    # convert state_command to byte
    state_command_byte = trial_state_command.to_bytes(1, 'big')
     # if True, extract condition nb and convert to byte
    if arg_keys_dict['conditionNum_channel'] == True:
        conditionNum_int = arg_values_dict["conditionNum"]
        conditionNum_byte = conditionNum_int.to_bytes(1, 'big')
    else: 
        conditionNum_int = 255
        conditionNum_byte = conditionNum_int.to_bytes(1, 'big')
    if [k for k, v in trial_state_commands_dict.items() if v == trial_state_command][0] == "sendsamples": # if trial_state_command = "sendsamples"
        zapit_com_bytes = [state_command_byte, arg_keys_byte, arg_values_byte, conditionNum_byte]
        zapit_com_ints = [trial_state_command, arg_keys_int, arg_values_int, conditionNum_int]
    else:
        zapit_com_bytes = [state_command_byte, (0).to_bytes(1, 'big'), (0).to_bytes(1, 'big'), (0).to_bytes(1, 'big')]  
        zapit_com_ints = [trial_state_command, 0, 0, 0]
    return zapit_com_bytes, zapit_com_ints

In [84]:
gen_Zapit_byte_tuple(trial_state_command = 1,
                     arg_keys_dict = {'conditionNum_channel': False, 'laser_channel': False, 'hardwareTriggered_channel': True, 'logging_channel': True, 'verbose_channel': True},
                     arg_values_dict = {'conditionNum': 101, 'laser_ON': True, 'hardwareTriggered_ON': False, 'logging_ON': False, 'verbose_ON': True})

([b'\x01', b'\x1c', b'\x12', b'\xff'], [1, 28, 18, 255])

In [80]:
def datetime_float_to_str(date_float):
    """
    Converts a MATLAB datenum value represented as a double-precision floating-point number
    to a string representation of the date and time in ISO format (YYYY-MM-DD HH:MM:SS).

    Parameters:
    date_float (float): A double-precision floating-point number representing the date and time
                        as a MATLAB datenum value.

    Returns:
    str: A string representation of the date and time in ISO format (YYYY-MM-DD HH:MM:SS).
    """
    # Convert the datenum value to a Unix timestamp
    unix_timestamp = (date_float - 719529) * 86400

    # Create a datetime object from the Unix timestamp
    datetime_str = str(datetime.datetime.fromtimestamp(unix_timestamp))

    return datetime_str

In [81]:
def parse_server_response(zapit_byte_tuple, datetime_double, message_type_byte, response_byte_tuple):
    """
    Parse server response and return a tuple with a datetime string and a tuple of integers representing the response of Zapit.

    Parameters:
    zapit_byte_tuple (tuple): The output of gen_Zapit_byte_tuple containing the expected message_type_byte as the first element.
    datetime_double (float): A double-precision floating-point number representing the date and time of the response,
                             or -1 if there was an error, or 1 if the server connected successfully.
    message_type_byte (bytes): A byte representing the message type of the response (e.g. sendSamples, stimConfLoaded etc.)
    response_byte_tuple (tuple): A tuple of bytes representing the server response  (response_byte_tuple[0] as conditionNumber & response_byte_tuple[1] as laserOn)

    Returns:
    tuple: A tuple containing a status string and a tuple of integers representing the server response.
           The status string can be one of the following:
           - 'Error': The datetime was -1, indicating an error.
           - 'Connected': The datetime was 1, indicating a successful connection.
           - 'Mismatch': The message type byte did not match the expected value in zapit_byte_tuple[0].
           - datetime_str: The datetime as a string, formatted using the datetime_float_to_str() function.
           The tuple of integers represents the status codes returned by the server.

    The function converts the byte strings in response_byte_tuple to a tuple of integers using a list comprehension.
    If the datetime is -1 or 1, the function returns a status string and the tuple of integers.
    If the message type byte does not match the expected value, the function returns 'Mismatch' and the tuple of integers.
    Otherwise, the function returns the datetime as a string and the tuple of integers.
    """
    server_response_ints = tuple(int(b[0]) for b in response_byte_tuple) # convert response_byte_tuple to a tuple of integers
    if datetime_double == -1:
        return('Error', server_response_ints)
    elif datetime_double == 1:
        return('Connected', server_response_ints)
    else:
        datetime_float64 = np.float64(datetime_double)
        datetime_str = datetime_float_to_str(datetime_float64)
        if message_type_byte != zapit_byte_tuple[0]: # check that Zapit is responding to the right message_type (e.g. sendSamples) else throw an error   
            return('Mismatch', server_response_ints)
        else: 
            pass
    return(datetime_str, server_response_ints)

In [86]:
parse_server_response(zapit_byte_tuple = [b'\x01', b'\x1e', b'\x12', b'e'],
                      datetime_double = 7.389744407802494e+05, # 7.389744407802494e+05
                      message_type_byte = b'\x01',
                      response_byte_tuple =  (b'\x01', b'\x02'))

('2023-03-29 11:34:43.413550', (1, 2))

In [ ]:
# tuple of [dataetime_float, message_byte, response_byte_tuple[0], response_byte_tuple[1], response_byte_tuple[2]]

In [93]:
def recover_dict_from_Zapit_tuple(zapit_com_ints):
    """
    Converts a list of integers representing a Zapit command into a tuple of values.

    Parameters:
    zapit_com_ints (list): A list of integers representing a Zapit command.

    Returns:
    tuple: A tuple containing the following values:
           - trial_state_command (int): An integer representing the trial state command.
                                        If it is 1, the command is "sendsamples".
           - arg_keys_dict (dict): A dictionary mapping integer keys to boolean values.
                                   The keys represent the channels in the Zapit command.
                                   The values are True if the channel is active, False otherwise.
           - arg_values_dict (dict): A dictionary mapping integer keys to boolean values.
                                     The keys represent the boolean arguments in the Zapit command.
                                     The values are True if the argument is True, False otherwise.
           - conditionNum (int): An integer representing the condition number.

    The function uses two reverse mapping dictionaries to convert the integer keys and arguments
    to string names, and a third dictionary to map 1/0 to True/False boolean values.
    If the trial state command is not "sendsamples", arg_keys_dict, arg_values_dict, and conditionNum
    are set to empty dictionaries and zero, respectively.
    """
    # reverse mapping dicts
    int_to_keys_dict = {1: "conditionNum_channel", 2: "laser_channel", 4: "hardwareTriggered_channel",
                        8: "logging_channel", 16: "verbose_channel"}

    int_to_bool_values_dict = {1: "conditionNum", 2: "laser_ON", 4: "hardwareTriggered_ON",
                               8: "logging_ON", 16: "verbose_ON"}

    # map 1/0 to True/False boolean values
    int_to_bool_dict = {1: True, 0: False}

    # extract trial_state_command
    trial_state_command = zapit_com_ints[0]

    if trial_state_command == 1: # if trial_state_command = "sendsamples"
        arg_keys_int = zapit_com_ints[1]
        arg_values_int = zapit_com_ints[2]
        conditionNum = zapit_com_ints[3]

        # extract arg_keys_dict from arg_keys_int
        arg_keys_dict = {}
        for key_int, key_name in int_to_keys_dict.items():
            if arg_keys_int & key_int:
                arg_keys_dict[key_name] = True
            else:
                arg_keys_dict[key_name] = False

        # extract arg_values_dict from arg_values_int
        arg_values_dict = {}
        for bool_int, bool_name in int_to_bool_values_dict.items():
            if arg_values_int & bool_int:
                arg_values_dict[bool_name] = True
            else:
                arg_values_dict[bool_name] = False

        # add conditionNum to arg_values_dict
        arg_values_dict["conditionNum"] = conditionNum

    else:
        # if trial_state_command != "sendsamples"
        arg_keys_dict = {}
        arg_values_dict = {}
        conditionNum = 0

    return trial_state_command, arg_keys_dict, arg_values_dict


In [96]:
zapit_byte_ints = [3, 1, 2, 3]
trial_state_command, arg_keys_dict, arg_values_dict = recover_dict_from_Zapit_tuple(zapit_byte_ints)

print(trial_state_command)  
print(arg_keys_dict)        
print(arg_values_dict)      

3
{}
{}
